# 📘 Semana 1 - Día 4: Encadenar Bloques

---

## 📋 Metadatos del Curso

- **Nivel:** Introductorio
- **Tiempo diario:** 20 minutos
- **Semana:** 1 — Fundamentos de Blockchain
- **Día:** 4 de 5 (+ 1 opcional)

---

## 🎯 Objetivo del Día

Al finalizar esta sesión, serás capaz de:
- Calcular el hash de un bloque completo usando todos sus campos
- Crear bloques encadenados mediante `previous_hash`
- Detectar alteraciones en la cadena mediante validación de hashes
- Comprender por qué la cadena es inmutable en la práctica

---

## 📖 Contenido Teórico

### El Concepto de Encadenamiento

El **encadenamiento** es el mecanismo que hace que blockchain sea "blockchain" (cadena de bloques). Cada bloque contiene el hash del bloque anterior, creando una cadena criptográfica.

**¿Cómo funciona?**

1. **Bloque Génesis** (Bloque 0):
   - No tiene bloque anterior
   - `previous_hash = "0"` (por convención)
   - Se calcula su hash usando todos sus campos

2. **Bloque 1**:
   - `previous_hash = hash del Bloque 0`
   - Se calcula su hash incluyendo el `previous_hash`

3. **Bloque 2**:
   - `previous_hash = hash del Bloque 1`
   - Y así sucesivamente...

### 🔗 ¿Por qué esto crea inmutabilidad?

Imagina que un atacante intenta modificar el **Bloque 1**:

```
ANTES:
Bloque 0 → hash: abc123
Bloque 1 → previous_hash: abc123, hash: def456
Bloque 2 → previous_hash: def456, hash: ghi789

DESPUÉS DE ALTERAR BLOQUE 1:
Bloque 0 → hash: abc123 (sin cambios)
Bloque 1 → previous_hash: abc123, hash: XXX999 (cambió!)
Bloque 2 → previous_hash: def456 ❌ (ya no coincide con XXX999)
```

**Consecuencia:** El `previous_hash` del Bloque 2 ya no coincide con el hash del Bloque 1. La cadena está rota y la alteración es evidente.

Para ocultar el cambio, el atacante tendría que:
1. Recalcular el hash del Bloque 1
2. Recalcular el hash del Bloque 2 (porque su `previous_hash` cambió)
3. Recalcular TODOS los bloques posteriores
4. Hacerlo más rápido que el resto de la red sigue agregando bloques nuevos

**En una blockchain real con miles de nodos y bloques, esto es computacionalmente imposible.**

---

## 💻 Código de Ejemplo: Crear Bloques Encadenados

Vamos a crear una función para calcular el hash de un bloque y luego encadenar varios bloques.

In [ ]:
import hashlib
import json
from datetime import datetime

def calcular_hash_bloque(bloque):
    """
    Calcula el hash SHA-256 de un bloque.
    
    Parámetros:
    - bloque: diccionario con los campos del bloque
    
    Retorna: hash hexadecimal del bloque
    """
    # 1. Crear una copia del bloque sin el campo 'hash' (para evitar recursión)
    bloque_sin_hash = {k: v for k, v in bloque.items() if k != 'hash'}
    
    # 2. Convertir el diccionario a string JSON (formato estándar)
    #    sort_keys=True asegura que el orden sea siempre el mismo
    bloque_string = json.dumps(bloque_sin_hash, sort_keys=True)
    
    # 3. Calcular el hash SHA-256
    return hashlib.sha256(bloque_string.encode('utf-8')).hexdigest()

def crear_bloque(index, data, previous_hash, nonce=0):
    """
    Crea un bloque con hash calculado automáticamente.
    """
    bloque = {
        'index': index,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'data': data,
        'previous_hash': previous_hash,
        'nonce': nonce,
        'hash': ''  # Placeholder temporal
    }
    # Calcular el hash real del bloque
    bloque['hash'] = calcular_hash_bloque(bloque)
    return bloque

# Crear el bloque génesis
bloque_0 = crear_bloque(
    index=0,
    data='Bloque Génesis',
    previous_hash='0'
)

print("=" * 70)
print("BLOQUE GÉNESIS")
print("=" * 70)
for campo, valor in bloque_0.items():
    print(f"{campo:15} : {valor}")
print("=" * 70)

### 🔍 Explicación del Código

1. **`json.dumps(bloque_sin_hash, sort_keys=True)`**  
   Convierte el diccionario a una cadena JSON. `sort_keys=True` garantiza que las claves estén siempre en el mismo orden, produciendo hashes consistentes.

2. **Excluir el campo `hash`**  
   No incluimos el campo `hash` en el cálculo del hash (sería circular). Solo usamos: index, timestamp, data, previous_hash, nonce.

3. **Asignación del hash**  
   Después de calcular el hash, lo asignamos al bloque.

---

## 🔗 Crear una Cadena de Bloques

Ahora creemos 2 bloques más, encadenados al bloque génesis.

In [ ]:
# Crear Bloque 1 (usa el hash del Bloque 0)
bloque_1 = crear_bloque(
    index=1,
    data='Alice → Bob: 10 BTC',
    previous_hash=bloque_0['hash']  # ← Enlace criptográfico
)

# Crear Bloque 2 (usa el hash del Bloque 1)
bloque_2 = crear_bloque(
    index=2,
    data='Bob → Charlie: 5 BTC',
    previous_hash=bloque_1['hash']  # ← Enlace criptográfico
)

# Mostrar la cadena
blockchain = [bloque_0, bloque_1, bloque_2]

print("\n" + "=" * 70)
print("BLOCKCHAIN COMPLETA (3 bloques)")
print("=" * 70)

for bloque in blockchain:
    print(f"\nBLOQUE #{bloque['index']}")
    print("-" * 70)
    print(f"  Data:          {bloque['data']}")
    print(f"  Previous Hash: {bloque['previous_hash'][:16]}...")
    print(f"  Hash:          {bloque['hash'][:16]}...")
    print("-" * 70)

print("\n🔗 Observa cómo el 'hash' de cada bloque se convierte en el")
print("   'previous_hash' del siguiente bloque. ¡Están encadenados!")
print("=" * 70)

---

## ✏️ Ejercicio Práctico

**Instrucciones:**  
Vamos a demostrar la inmutabilidad de la blockchain.

**Tareas:**
1. Altera el campo `data` del **Bloque 1** (cambia "Alice" por "Mallory")
2. Recalcula el hash del Bloque 1 con los datos alterados
3. Compara el nuevo hash con el `previous_hash` del Bloque 2
4. Escribe una función simple `validar_cadena()` que detecte si la cadena está rota

**💡 Pista:** La validación debe verificar que el `previous_hash` de cada bloque coincida con el `hash` del bloque anterior.

In [ ]:
# TU CÓDIGO AQUÍ

# Paso 1: Alterar el Bloque 1
print("ANTES DE ALTERAR:")
print(f"Bloque 1 - Data: {bloque_1['data']}")
print(f"Bloque 1 - Hash: {bloque_1['hash'][:16]}...")
print(f"Bloque 2 - Previous Hash: {bloque_2['previous_hash'][:16]}...")
print()

# Alterar los datos
bloque_1['data'] = # TU CÓDIGO: cambia "Alice" por "Mallory"

# Paso 2: Recalcular el hash
nuevo_hash_bloque1 = # TU CÓDIGO: usa calcular_hash_bloque()

print("DESPUÉS DE ALTERAR:")
print(f"Bloque 1 - Data: {bloque_1['data']}")
print(f"Bloque 1 - Nuevo Hash: {nuevo_hash_bloque1[:16]}...")
print(f"Bloque 2 - Previous Hash: {bloque_2['previous_hash'][:16]}...")
print()

# Paso 3: Comparar
if nuevo_hash_bloque1 == bloque_2['previous_hash']:
    print("✅ La cadena es válida")
else:
    print("❌ ¡CADENA ROTA! Los hashes no coinciden.")
    print("   La alteración ha sido detectada.")

---

## ✅ Solución — NO VER hasta intentar

<details>
<summary>Haz clic para ver la solución</summary>

In [ ]:
# SOLUCIÓN COMENTADA

import hashlib
import json
from datetime import datetime

# Funciones auxiliares (copiar del ejemplo anterior)
def calcular_hash_bloque(bloque):
    bloque_sin_hash = {k: v for k, v in bloque.items() if k != 'hash'}
    bloque_string = json.dumps(bloque_sin_hash, sort_keys=True)
    return hashlib.sha256(bloque_string.encode('utf-8')).hexdigest()

def crear_bloque(index, data, previous_hash, nonce=0):
    bloque = {
        'index': index,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'data': data,
        'previous_hash': previous_hash,
        'nonce': nonce,
        'hash': ''
    }
    bloque['hash'] = calcular_hash_bloque(bloque)
    return bloque

# Recrear la blockchain
bloque_0 = crear_bloque(0, 'Bloque Génesis', '0')
bloque_1 = crear_bloque(1, 'Alice → Bob: 10 BTC', bloque_0['hash'])
bloque_2 = crear_bloque(2, 'Bob → Charlie: 5 BTC', bloque_1['hash'])

# DEMOSTRACIÓN DE ALTERACIÓN
print("=" * 70)
print("DEMOSTRACIÓN: DETECCIÓN DE ALTERACIONES")
print("=" * 70)

print("\n📋 ESTADO ORIGINAL:")
print(f"Bloque 1 - Data: {bloque_1['data']}")
print(f"Bloque 1 - Hash: {bloque_1['hash']}")
print(f"Bloque 2 - Previous Hash: {bloque_2['previous_hash']}")
print(f"¿Coinciden? {bloque_1['hash'] == bloque_2['previous_hash']} ✅")

# Alterar el Bloque 1 (simular ataque)
print("\n🚨 ALTERANDO BLOQUE 1...")
bloque_1['data'] = 'Mallory → Bob: 10 BTC'  # Cambio malicioso

# Recalcular el hash del bloque alterado
nuevo_hash_bloque1 = calcular_hash_bloque(bloque_1)

print("\n📋 ESTADO DESPUÉS DE ALTERACIÓN:")
print(f"Bloque 1 - Data: {bloque_1['data']}")
print(f"Bloque 1 - Nuevo Hash: {nuevo_hash_bloque1}")
print(f"Bloque 2 - Previous Hash: {bloque_2['previous_hash']}")
print(f"¿Coinciden? {nuevo_hash_bloque1 == bloque_2['previous_hash']} ❌")

print("\n" + "=" * 70)
print("CONCLUSIÓN:")
print("El hash del Bloque 1 cambió completamente.")
print("El 'previous_hash' del Bloque 2 ya no coincide.")
print("¡La alteración es EVIDENTE! La cadena está rota.")
print("=" * 70)

# BONUS: Función de validación
def validar_cadena(blockchain):
    """
    Valida que todos los bloques estén correctamente encadenados.
    
    Retorna: True si la cadena es válida, False si está rota
    """
    for i in range(1, len(blockchain)):
        bloque_actual = blockchain[i]
        bloque_anterior = blockchain[i - 1]
        
        # Verificar que el previous_hash coincida con el hash del bloque anterior
        if bloque_actual['previous_hash'] != bloque_anterior['hash']:
            print(f"❌ Error en Bloque {i}: previous_hash no coincide")
            return False
        
        # Verificar que el hash del bloque sea correcto
        hash_calculado = calcular_hash_bloque(bloque_actual)
        if bloque_actual['hash'] != hash_calculado:
            print(f"❌ Error en Bloque {i}: hash ha sido alterado")
            return False
    
    return True

# Probar la validación
blockchain_alterada = [bloque_0, bloque_1, bloque_2]
print("\n🔍 VALIDANDO BLOCKCHAIN ALTERADA:")
es_valida = validar_cadena(blockchain_alterada)
if es_valida:
    print("✅ Blockchain válida")
else:
    print("❌ Blockchain inválida - se detectaron alteraciones")

</details>

---

## 📝 Autoevaluación — Respuestas al final de la sección

### Pregunta 1
**¿Qué campo de un bloque crea el enlace criptográfico con el bloque anterior?**

A) `index`  
B) `timestamp`  
C) `previous_hash`  
D) `nonce`  

### Pregunta 2
**Verdadero o Falso: Si alteras un bloque antiguo en la cadena, solo necesitas recalcular el hash de ese bloque para ocultar el cambio.**

A) Verdadero  
B) Falso  

### Pregunta 3
**¿Por qué es computacionalmente imposible alterar bloques antiguos en una blockchain real?**

A) Los bloques están encriptados con contraseñas  
B) Tendrías que recalcular todos los bloques posteriores más rápido que la red agrega nuevos bloques  
C) Los bloques se almacenan en servidores seguros  
D) Solo el administrador puede modificar bloques  

---

### 🔑 Respuestas

1. **Respuesta: C**  
   *Explicación:* El campo `previous_hash` contiene el hash del bloque anterior, creando el enlace criptográfico que da nombre a "blockchain" (cadena de bloques).

2. **Respuesta: B (Falso)**  
   *Explicación:* Si alteras un bloque, su hash cambia. Esto rompe el `previous_hash` del siguiente bloque, por lo que tendrías que recalcular TODOS los bloques posteriores, no solo el alterado.

3. **Respuesta: B**  
   *Explicación:* Para ocultar una alteración, necesitarías recalcular todos los bloques posteriores (lo cual requiere resolver los desafíos de minería) más rápido que miles de nodos honestos siguen agregando bloques nuevos. Esto requiere más del 51% del poder computacional de la red.

---

## 🎯 Resumen del Día 4

Hoy aprendiste:
- ✅ Cómo calcular el hash de un bloque completo usando `json.dumps()` y `hashlib`
- ✅ Cómo encadenar bloques mediante el campo `previous_hash`
- ✅ Por qué alterar un bloque antiguo rompe toda la cadena posterior
- ✅ Cómo implementar una función de validación para detectar alteraciones

### 🔐 Concepto Clave: Inmutabilidad Práctica
La blockchain no es técnicamente "imposible de modificar", pero el costo computacional de alterar bloques antiguos y recalcular toda la cadena posterior es tan alto que se vuelve **económicamente inviable** en redes grandes.

### 📚 Próximo Paso
En el **Día 5**, consolidaremos todo lo aprendido. Crearás una blockchain completa con múltiples bloques, implementarás validación robusta, y reflexionarás sobre las implicaciones de esta tecnología.

---

**¡Excelente trabajo! 🚀**  
*Concepto clave: El encadenamiento criptográfico convierte la historia en algo verificable e inmutable.*